In [1]:
import logging
logging.basicConfig(filename = '../logging/BrainNetsLog.log', 
                    filemode='w', 
                    format = '%(asctime)s %(name)s %(lineno)d %(funcName)s %(levelname)s:%(message)s', 
                    level = logging.INFO)
logger = logging.getLogger('Jupyter Notebook')

In [2]:
import BrainNets as bn
reload(bn)
import dataOperator as do
reload(do)
import models.SectorNet as sn

import time
import theano
import numpy as np
import SimpleITK as sitk
import matplotlib.pyplot as plt
from tqdm import tqdm_notebook

%matplotlib inline 

/home/mountain/anaconda3/envs/biomediclasagne/lib/python2.7/site-packages/theano/gpuarray/dnn.py:135: UserWarning: Your cuDNN version is more recent than Theano. If you encounter problems, try updating Theano or downgrading cuDNN to version 5.1.
  warnings.warn("Your cuDNN version is more recent than "
Using cuDNN version 6020 on context None
Mapped name None to device cuda0: GeForce GTX 750 Ti (0000:01:00.0)


In [3]:
sectorNet = sn.SectorNet('/home/mountain/Desktop/BrainNets/code/configs/modelConfigs/SectorNetModel.cfg')
trainFunc = sectorNet.trainFunction()
valAndTestFunc = sectorNet.valAndTestFunction()

In [4]:
trainCubes = do.cubesGetor(cubeDirectory = '../data/tryGenerateCubeData/', 
                           sourceDataDirectory = '../data/SimpleExploreSomeData/', 
                           subStructure = 'non-enhancing',
                           cubeSize = 17,
                           trainOrVal = 'train',
                           onTheFly = True)
valCubes = do.cubesGetor(cubeDirectory = '../data/tryGenerateCubeData/', 
                         sourceDataDirectory = '../data/SimpleExploreSomeData/',
                         subStructure = 'non-enhancing',
                         cubeSize = 17,
                         onTheFly = True, 
                         trainOrVal = 'val')

In [5]:
numEpochs = 4

In [6]:
trainingLoss = []
validationLoss = []
validationACC = []

In [ ]:
for epoch in range(numEpochs):
    logger.info('{} th of {} epochs'.format(epoch, numEpochs))
    
    trainError = 0
    startTime = time.time()
    trainBatch = 0
    
#     for batch in synthsisTrain.generateSynthesisData(1000, group0 = (0,5), group1 = (1, 5)):
    for batch in trainCubes.fetchCubes(batchSize = 100, parallel = True):
        inputs, targets = batch
        trainLoss, trainACC = trainFunc(inputs, targets / 3)

        trainError += trainLoss
        trainBatch += 1
        
        print trainLoss, trainACC

    valError = 0
    valACC = 0
    valBatches = 0
        
#     for batch in synthsisVal.generateSynthesisData(1000, group0 = (1,5), group1 = (1, 5)):
    for batch in valCubes.fetchCubes(batchSize = 100, parallel = True):
        inputs, targets = batch
        targets = targets
        valAndTestLoss, valAndTestACC = valAndTestFunc(inputs, targets)
        valError += valAndTestLoss
        valACC += valAndTestACC
        valBatches += 1
        
    print "Epoch {} of {} took {:.3f}s".format(epoch + 1, numEpochs, time.time() - startTime)
    print "  training loss:\t\t{:.6f}".format(trainError / trainBatch)
    print "  validation loss:\t\t{:.6f}".format(valError / valBatches)
    print "  validation accuracy:\t\t{:.2f} %".format(valACC / valBatches * 100)
    
    trainingLoss.append(trainError / trainBatch)
    validationLoss.append(valError / valBatches)
    validationACC.append(valACC / valBatches * 100)

0.706139922142 0.579999983311
0.535633087158 0.670000016689
0.438118547201 0.810000002384
0.418981552124 0.920000016689
0.392769098282 0.930000007153
0.405160784721 0.910000026226
0.396758198738 0.889999985695
0.357275158167 0.939999997616
0.369100987911 0.939999997616
0.362949490547 0.949999988079
0.406369775534 0.930000007153
0.384882211685 0.910000026226
0.384325861931 0.920000016689
0.349830120802 0.980000019073
0.398659169674 0.870000004768


Process PoolWorker-64:
Process PoolWorker-63:
Traceback (most recent call last):
Process PoolWorker-62:
Process PoolWorker-61:
Traceback (most recent call last):
  File "/home/mountain/anaconda3/envs/biomediclasagne/lib/python2.7/site-packages/multiprocess/process.py", line 258, in _bootstrap
  File "/home/mountain/anaconda3/envs/biomediclasagne/lib/python2.7/site-packages/multiprocess/process.py", line 258, in _bootstrap
    self.run()
Traceback (most recent call last):
    self.run()
  File "/home/mountain/anaconda3/envs/biomediclasagne/lib/python2.7/site-packages/multiprocess/process.py", line 114, in run
  File "/home/mountain/anaconda3/envs/biomediclasagne/lib/python2.7/site-packages/multiprocess/process.py", line 114, in run
    self._target(*self._args, **self._kwargs)
  File "/home/mountain/anaconda3/envs/biomediclasagne/lib/python2.7/site-packages/multiprocess/process.py", line 258, in _bootstrap
Traceback (most recent call last):
  File "/home/mountain/anaconda3/envs/biomedic

In [ ]:
plt.plot(trainingLoss)
plt.ylabel('trainingLoss')
plt.show()

In [ ]:
plt.plot(validationLoss)
plt.ylabel('validationLoss')
plt.show()

In [ ]:
plt.plot(validationACC)
plt.ylabel('validationACC')
plt.show()

In [ ]:
theano.printing.debugprint(trainFunc)

In [ ]:
%time do.normalizeDataSet(dataPath = '../data/trainParallelNorm/', normalizedDataDir = '../data/outPutPara1/',parallel = False)

In [ ]:
%time do.normalizeDataSet(dataPath = '../data/trainParallelNorm/', normalizedDataDir = '../data/outPutPara2/',parallel = True)

In [ ]:
do.makeCubesOnTheFly(dataPath = '../data/SimpleExploreSomeData/', 
             cubeDirectory = '../data/tryGenerateCubeData1/',
             cubeSize = 33, 
             parallel = False)

In [ ]:
do.makeCubesOnTheFly(dataPath = '../data/SimpleExploreSomeData/', 
             cubeDirectory = '../data/tryGenerateCubeData/',
             cubeSize = 33, 
             parallel = True)

In [ ]:
synthsisTrain = do.cubesGetor(cubeDirectory = '../data/tryGenerateCubeData/', batchSize = 1000)
synthsisVal = do.cubesGetor(cubeDirectory = '../data/tryGenerateCubeData/', trainOrVal = 'val', batchSize = 1000)

In [ ]:
startTime = time.time()
for inputs, targets in trainCubes.fetchCubes(batchSize = 100, parallel = False):
    print time.time() - startTime
    startTime = time.time()

In [ ]:
startTime = time.time()
for inputs, targets in trainCubes.fetchCubes(batchSize = 100, parallel = True):
    print time.time() - startTime
    startTime = time.time()